# Creating a Model and Running the Training Steps

In [ ]:
#-------Loading Necessary Packages----------#
from keras.preprocessing.image import ImageDataGenerator,load_img,img_to_array
from keras.models import Sequential,load_model

import os
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
#from keras import callbacks
#from PIL.Image import image


# dimensions of our images.
img_width, img_height = 360, 563
#----Splitting the Training Data into Validation Data and Training Data and providing Necessary Parameters
train_data_dir = ("images/Train Directory")
validation_data_dir = ("images/Train Directory/")
nb_train_samples = 3370
nb_validation_samples = 3370
epochs = 20
batch_size = 16
classes_num = 13

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Dense(classes_num, activation='softmax'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

"""
Tensorboard log
"""
log_dir = './tf-log/'


model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples //batch_size)



In [ ]:
target_dir = './models/'
if not os.path.exists(target_dir):
  os.mkdir(target_dir)
model.save('./models/model.h5')
model.save_weights('./models/weights.h5')

In [ ]:
import numpy as np
from sklearn.metrics import classification_report,confusion_matrix
#Confution Matrix and Classification Report
Y_pred = model.predict_generator(validation_generator, nb_validation_samples // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))
print('Classification Report')
target_names = ['abstract','Checked','Colourblock','floral','graphic','Melange','Patterned','polka dots','Predicted','Printed','solid','striped','typography']
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))

In [ ]:
cm = (confusion_matrix(validation_generator.classes, y_pred))
import matplotlib.pyplot as plt
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
import itertools
cm_plot_labels = ['abstract','Checked','Colourblock','floral','graphic','Melange','Patterned','polka dots','Predicted','Printed','solid','striped','typography']
plot_confusion_matrix(cm,cm_plot_labels,title="Confusion Matrix for Fynd")

In [ ]:
def predict(file):
  x = load_img(file, target_size=(img_width,img_height))
  x = img_to_array(x)
  x = np.expand_dims(x, axis=0)
  array = model.predict(x)
  result = array[0]
  answer = np.argmax(result)
  if answer == 0:  
    print("abstract")
  elif answer ==1:
    print("Checked")
  elif answer ==2:
    print("ColourBlock")
  elif answer ==3:
    print("floral")
  elif answer ==4:
    print("graphic")
  elif answer ==5:
    print("Melange")
  elif answer ==6:
    print("Patterned")
  elif answer ==7:
    print("polka dots")
  elif answer ==8:
    print("Predicted")
  elif answer ==9:
    print("Printed")
  elif answer ==10:
    print("solid")
  elif answer ==11:
    print("striped")
  elif answer ==12:
    print("typography")

  return answer

In [ ]:
import pandas as pd
output = pd.DataFrame(columns = ['Image','Predicted_Category','Category_Name'])

Images = []
category = []

for file in os.listdir('./images/Test Directory/'):
    Images.append('./images/Test Directory/'+ file)
    the_category = (predict('./images/Test Directory/'+ file))
    category.append(the_category)
    
output['Image'] = Images
output['Predicted_Category'] = category

In [ ]:
for i in range(len(output)):
    if output['Predicted_Category'][i] == 0:
        output['Category_Name'][i] = 'abstract'
    elif output['Predicted_Category'][i] ==1:
        output['Category_Name'][i] = "Checked"
    elif output['Predicted_Category'][i] ==2:
        output['Category_Name'][i] = "ColourBlock"
    elif output['Predicted_Category'][i] ==3:
        output['Category_Name'][i] = "floral"
    elif output['Predicted_Category'][i] ==4:
        output['Category_Name'][i] = "graphic"
    elif output['Predicted_Category'][i] ==5:
        output['Category_Name'][i] = "Melange"
    elif output['Predicted_Category'][i] ==6:
        output['Category_Name'][i] = "Patterned"
    elif output['Predicted_Category'][i] ==7:
        output['Category_Name'][i] = "polka dots"
    elif output['Predicted_Category'][i] ==8:
        output['Category_Name'][i] = "Predicted"
    elif output['Predicted_Category'][i] ==9:
        output['Category_Name'][i] = "Printed"
    elif output['Predicted_Category'][i] ==10:
        output['Category_Name'][i] = "solid"
    elif output['Predicted_Category'][i] ==11:
        output['Category_Name'][i] = "striped"
    elif output['Predicted_Category'][i] ==12:
        output['Category_Name'][i] = "typography"
        

In [ ]:
output

In [ ]:
output.to_csv('test_result.csv')

In [ ]:
score= model.evaluate_generator(validation_generator,nb_validation_samples // batch_size, workers=1)

In [ ]:
accuracy = score[1]

In [ ]:
print("The model predicted at an accuracy of:",accuracy)
print("The model predicted with an loss of:",score[0])

In [ ]:
predict("./images/Train Directory/abstract/66081.jpg")